# Training Regression Models using sklearn pipelines

## Importing Libraries

In [1]:
#importing Libraries
import pandas as pd
# import dvc.api
import os
import sys
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import mlflow
#import local libraries
#Adding scripts path
sys.path.append(os.path.abspath(os.path.join('..')))
from scripts.data_loader import load_df_from_csv
from scripts.ML_modelling_utils import *
#importing dvc_data_loader script
# from scripts.dvc_data_loader import *


## Loading Clean Data

In [2]:
clean_data = load_df_from_csv('../data/train.csv')
y_values = clean_data['Sales']
x_values = clean_data.drop(['Sales'],axis=1)

## Training using Random Forest Regressor

In [3]:
# Splitting Data (60,20,20)
x_train, x_test, y_train, y_test = train_test_split(x_values, y_values, test_size=0.2, random_state=42)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [4]:
mlflow.autolog()
# Create a based model
rf = RandomForestRegressor()
with mlflow.start_run() as run:
    rf.fit(x_train, y_train)

    train_score = rf.score(x_train, y_train)
    valid_score = rf.score(x_valid,y_valid)
    valid_metrics = calculate_metrics(y_valid,rf.predict(x_valid))
    test_score = rf.score(x_test,y_test)
    test_metrics = calculate_metrics(y_test, rf.predict(x_test))

    mlflow.log_metric("Valid Score", valid_score)
    mlflow.log_metrics(valid_metrics)
    mlflow.log_metric("Test Score", test_score)
    mlflow.log_metrics(test_metrics)


2021/07/29 11:17:55 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2021/07/29 11:17:55 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.


In [5]:
train_score

0.9918217208283953

In [6]:
valid_score

0.9464088698187895

In [7]:
valid_metrics

{'RMSE Score': 0.23207756030571391,
 'R2_Squared': 0.9464088698187895,
 'MAE Score': 0.13363749553641205}

In [8]:
test_metrics


{'RMSE Score': 0.23242887038376997,
 'R2_Squared': 0.9458018010617835,
 'MAE Score': 0.13350047867580667}

In [9]:
features = pd.DataFrame()
features["Feature"] = x_train.columns
features["Importance"] = rf.feature_importances_
features.sort_values(by='Importance', ascending=False)


,Feature,Importance
7,Open,0.460141
15,CompetitionDistance,0.161633
8,Promo,0.073437
17,CompetitionOpenSinceYear,0.050305
16,CompetitionOpenSinceMonth,0.049603
0,DayOfWeek,0.033223
19,Promo2SinceWeek,0.024841
13,StoreType,0.024464
5,Day,0.022929
3,Month,0.019423


In [10]:
# with mlflow.start_run():
#     mlflow.log_parameter('')
#     mlflow.log_metric('')
#     mlflow.sklearn.log_model(rf,'model')

In [11]:
# Create the parameter grid based on the results of random search
param_grid = {
    'bootstrap': [True],
    'criterion': ['mse'],
    'max_depth': [80, 90],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4],
    'min_samples_split': [8, 10],
    'n_estimators': [100, 200]
}

rf2 = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator=rf2, param_grid=param_grid,
                           cv=3, n_jobs=-1, verbose=2)


In [12]:
# Fit the grid search to the data
# grid_search.fit(x_train, y_train)

In [13]:
# grid_search.best_params_

# best_grid = grid_search.best_estimator_
